In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
BASE_PATH = "/content/drive/MyDrive/DIP_OEL/PH2Dataset"

In [4]:
import os
print(os.listdir(BASE_PATH)[:5])

['Readme.txt', 'PH2_dataset.txt', 'PH2_dataset.xlsx', 'PH2 Dataset images']


PROJECT START

Task 1 — Lesion Segmentation & Mask Generation

🔹 Required Libraries

In [5]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

🔹 Helper Functions (Image Processing Pipeline) 1️⃣ Image Enhancement

In [6]:
def enhance_image(img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    l = clahe.apply(l)
    enhanced = cv2.merge((l, a, b))
    return cv2.cvtColor(enhanced, cv2.COLOR_LAB2BGR)

2️⃣ Hair Removal (Black-hat + Inpainting)

In [7]:
def remove_hairs(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (17,17))
    blackhat = cv2.morphologyEx(gray, cv2.MORPH_BLACKHAT, kernel)
    _, mask = cv2.threshold(blackhat, 10, 255, cv2.THRESH_BINARY)
    return cv2.inpaint(img, mask, 1, cv2.INPAINT_TELEA)

3️⃣ Lesion Segmentation

In [8]:
def segment_lesion(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)

    _, binary = cv2.threshold(
    blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
)


    # 1️⃣ CLOSE — lesion boundaries connect karne ke liye
    kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,7))
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel_close)

    # 2️⃣ FILL HOLES — gaps remove karne ke liye
    h, w = binary.shape
    floodfill = binary.copy()
    mask = np.zeros((h+2, w+2), np.uint8)
    cv2.floodFill(floodfill, mask, (0,0), 255)
    floodfill_inv = cv2.bitwise_not(floodfill)
    filled = binary | floodfill_inv

    return filled

🔹 Main Mask Generation Loop

In [9]:
BASE_PATH = "/content/drive/MyDrive/DIP_OEL/PH2Dataset/PH2 Dataset images"
SAVE_PATH = "/content/drive/MyDrive/DIP_OEL/PH2Dataset/generated_masks"

os.makedirs(SAVE_PATH, exist_ok=True)
os.makedirs(SAVE_PATH, exist_ok=True)

y_true_all, y_pred_all = [], []

for folder in tqdm(os.listdir(BASE_PATH)):
    folder_path = os.path.join(BASE_PATH, folder)

    if not os.path.isdir(folder_path):
        continue

    img_path = os.path.join(
        folder_path, f"{folder}_Dermoscopic_Image"
    )
    gt_path = os.path.join(
        folder_path, f"{folder}_lesion"
    )

    img_file = os.listdir(img_path)[0]
    gt_file  = os.listdir(gt_path)[0]

    img = cv2.imread(os.path.join(img_path, img_file))
    gt  = cv2.imread(os.path.join(gt_path, gt_file), 0)

    if img is None or gt is None:
        continue

    img = enhance_image(img)
    img = remove_hairs(img)
    pred_mask = segment_lesion(img)

    cv2.imwrite(
        os.path.join(SAVE_PATH, f"{folder}_mask.png"),
        pred_mask
    )

    y_true_all.extend((gt > 0).flatten())
    y_pred_all.extend((pred_mask > 0).flatten())

100%|██████████| 200/200 [05:54<00:00,  1.77s/it]


In [10]:
import os

path_to_verify = '/content/drive/MyDrive/DIP_OEL/PH2Dataset/PH2 Dataset images'

if os.path.exists(path_to_verify):
    if os.path.isdir(path_to_verify):
        print(f"Path '{path_to_verify}' exists and is a directory.")
        print("Contents (first 10 items):", os.listdir(path_to_verify)[:5])
    else:
        print(f"Path '{path_to_verify}' exists but is not a directory.")
else:
    print(f"Path '{path_to_verify}' does not exist.")

Path '/content/drive/MyDrive/DIP_OEL/PH2Dataset/PH2 Dataset images' exists and is a directory.
Contents (first 10 items): ['IMD004', 'IMD009', 'IMD002', 'IMD003', 'IMD008']


📊 3. Segmentation Evaluation (Dataset-Level)

In [11]:
cm = confusion_matrix(y_true_all, y_pred_all)
TN, FP, FN, TP = cm.ravel()

accuracy = (TP + TN) / (TP + TN + FP + FN)
sensitivity = TP / (TP + FN)
specificity = TN / (TN + FP)

print("Confusion Matrix:\n", cm)
print(f"Accuracy     : {accuracy:.4f}")
print(f"Sensitivity  : {sensitivity:.4f}")
print(f"Specificity  : {specificity:.4f}")

Confusion Matrix:
 [[ 2133754 57593928]
 [  933741 27501900]]
Accuracy     : 0.3361
Sensitivity  : 0.9672
Specificity  : 0.0357


Step 1: Import libraries and mount drive

In [12]:
from google.colab import drive
import os
import cv2
import numpy as np
import pandas as pd

drive.mount('/content/drive')

BASE_PATH = "/content/drive/MyDrive/DIP_OEL/PH2Dataset/PH2 Dataset images"
SAVE_PATH = "/content/drive/MyDrive/DIP_OEL/PH2Dataset/Masks"  # jahan masks saved hain
LABEL_TXT = "/content/drive/MyDrive/DIP_OELT/PH2Dataset/PH2_dataset.txt"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Load labels from .txt file

In [14]:
LABEL_TXT = "/content/drive/MyDrive/DIP_OEL/PH2Dataset/PH2_dataset.txt"
label_dict = {}

with open(LABEL_TXT, "r") as f:
    lines = f.readlines()

for line in lines:
    if line.strip() == "":
        continue

    parts = line.strip().split("||")
    parts = [p.strip() for p in parts if p.strip() != ""]

    # Skip header
    if parts[0] == "Name":
        continue

    img_id = parts[0]

    try:
        melanoma_val = parts[-1].split()[0]
        melanoma_val = int(melanoma_val)
        label = 1 if melanoma_val > 0 else 0
        label_dict[img_id] = label
    except:
        continue

print("Total labels loaded:", len(label_dict))
print("Sample:", list(label_dict.items())[:10])

Total labels loaded: 212
Sample: [('IMD003', 1), ('IMD009', 1), ('IMD016', 1), ('IMD022', 1), ('IMD024', 1), ('IMD025', 1), ('IMD035', 1), ('IMD038', 1), ('IMD042', 1), ('IMD044', 1)]


 Define feature extraction function

In [15]:
def extract_features(img, mask):
    features = []

    # strict binary mask
    mask = (mask > 0).astype(np.uint8)

    for i in range(3):  # BGR channels
        channel = img[:, :, i]
        lesion_pixels = channel[mask == 1]

        if lesion_pixels.size < 50:   # too small region
            features.extend([0, 0])
        else:
            features.extend([
                np.mean(lesion_pixels) / 255.0,
                np.std(lesion_pixels) / 255.0
            ])
    return features



Load images, masks, extract features

In [16]:
from google.colab import drive
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

# ----------------------
# Step 0: Mount Drive
# ----------------------
drive.mount('/content/drive')

# ----------------------
# Step 1: Define Paths
# ----------------------
BASE_PATH = "/content/drive/MyDrive/DIP_OEL/PH2Dataset/PH2 Dataset images"
SAVE_PATH = "/content/drive/MyDrive/DIP_OEL/PH2Dataset/generated_masks"
LABEL_TXT = "/content/drive/MyDrive/DIP_OEL/PH2Dataset/PH2_dataset.txt"

# ----------------------
# Step 2: Load Labels from .txt
# ----------------------
label_dict = {}

with open(LABEL_TXT, "r") as f:
    lines = f.readlines()

for line in lines:
    if line.strip() == "":
        continue

    parts = line.strip().split("||")
    parts = [p.strip() for p in parts if p.strip() != ""]

    # Skip header
    if parts[0] == "Name":
        continue

    img_id = parts[0]

    try:
        melanoma_val = parts[-1].split()[0]
        melanoma_val = int(melanoma_val)
        label = 1 if melanoma_val > 0 else 0
        label_dict[img_id] = label
    except:
        continue

print("Total labels loaded:", len(label_dict))
print("Sample labels:", list(label_dict.items())[:10])

# ----------------------
# Step 3: Define Feature Extraction
# ----------------------
def extract_features(img, mask):
    """
    Extract simple features from masked image:
    mean and std for each channel (BGR)
    """
    features = []

    # Ensure mask is binary
    _, mask_bin = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)

    for i in range(3):  # B, G, R channels
        masked_pixels = img[:, :, i][mask_bin > 0]
        if masked_pixels.size == 0:
            features.extend([0, 0])
        else:
            features.extend([masked_pixels.mean(), masked_pixels.std()])
    return features  # total 6 features

# ----------------------
# Step 4: Load Images, Masks and Extract Features
# ----------------------
X = []
y = []

missing_masks = []
missing_images = []

for folder in os.listdir(BASE_PATH):
    folder_path = os.path.join(BASE_PATH, folder)

    if folder not in label_dict:
        continue

    # Mask path
    mask_path = os.path.join(SAVE_PATH, f"{folder}_mask.png")
    if not os.path.exists(mask_path):
        missing_masks.append(folder)
        continue

    # Image path
    img_folder = os.path.join(folder_path, f"{folder}_Dermoscopic_Image")
    if not os.path.exists(img_folder):
        missing_images.append(folder)
        continue

    img_files = os.listdir(img_folder)
    if len(img_files) == 0:
        missing_images.append(folder)
        continue

    img_file = img_files[0]
    img = cv2.imread(os.path.join(img_folder, img_file))
    mask = cv2.imread(mask_path, 0)  # grayscale

    if img is None or mask is None:
        missing_images.append(folder)
        continue

    feats = extract_features(img, mask)
    X.append(feats)
    y.append(label_dict[folder])

X = np.array(X)
y = np.array(y)

print("Features shape:", X.shape)
print("Labels shape:", y.shape)
print("Missing masks:", len(missing_masks))
print("Missing images:", len(missing_images))



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total labels loaded: 212
Sample labels: [('IMD003', 1), ('IMD009', 1), ('IMD016', 1), ('IMD022', 1), ('IMD024', 1), ('IMD025', 1), ('IMD035', 1), ('IMD038', 1), ('IMD042', 1), ('IMD044', 1)]
Features shape: (200, 6)
Labels shape: (200,)
Missing masks: 0
Missing images: 0


Train/Test Split Example

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

Classifier:

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Train Random Forest
clf = RandomForestClassifier(
    n_estimators=50,
    max_depth=5,
    min_samples_split=5,
    random_state=42
)
clf.fit(X_train, y_train)

# Predict
y_pred = clf.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 1.0
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        40

    accuracy                           1.00        40
   macro avg       1.00      1.00      1.00        40
weighted avg       1.00      1.00      1.00        40

